In [23]:
%cd /hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning/

import torch
import json
from pathlib import Path

res_dict = {}
predict_file = "inference/llama3_nlp_new_batch_rw_vanilla/all_res.json"
ground_truth_file = "dataset/NLP_rw_new_batch/instruction/test_data_list_1500_8192_ans_gt_256_llama3.pt"

predict_res = json.load(open(predict_file))
predict_res = {int(list(entry.keys())[0]):list(entry.values())[0] for entry in predict_res}

ground_truths = torch.load(ground_truth_file)
for data_entry in ground_truths:
    if data_entry["id"] in predict_res:
        res_dict[data_entry["id"]] = {
            # "predict": predict_res[data_entry["id"]],
            "predict": predict_res[data_entry["id"]][0]['generated_text'][1]["content"],
            "ground_truth": data_entry["ground_truth"]
        }

print(len(res_dict))

res_dict = {k: v for k, v in res_dict.items() if "ground_truth" in v}
print(len(res_dict))

/hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning
476
476


In [15]:
predict_res

{0: 'Analyzing representations and neurons is a crucial aspect of understanding the inner workings of neural networks, particularly in the context of natural language processing. Recent studies have delved into the linguistic knowledge and transferability of contextual word representations, revealing that these representations encode valuable and transferable language features @CITE[2]@. Furthermore, research has shown that pre-trained language models are aware of phrases and can capture the syntactic notion of constituency @CITE[3]@. The representational power of neural machine translation models has also been investigated, with findings indicating that these models learn a non-trivial amount of linguistic information, including word morphology, part-of-speech information, and lexical semantics @CITE[4]@, @CITE[6]@. Additionally, the presence of expert units in transformer models has been found to be correlated with the generalization power of these models, and can be leveraged to imp

In [24]:
import numpy as np
# calculate the BLEU score with HUGGINGFACE datasets
import evaluate

bleu_score_metric = evaluate.load("bleu")
for id, res in res_dict.items():
    bleu_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bleu_score = bleu_score_metric.compute()
print("bleu_score: ", bleu_score)

rouge_score_metric = evaluate.load("rouge")
for id, res in res_dict.items():
    rouge_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
rouge_score = rouge_score_metric.compute()
print("rouge_score: ", rouge_score)

bert_score_metric = evaluate.load("bertscore")
for id, res in res_dict.items():
    bert_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bert_score = bert_score_metric.compute(model_type="distilbert-base-uncased")
print("bert_score: ", np.mean(bert_score['precision']))

bleu_score:  {'bleu': 0.053804291058031216, 'precisions': [0.3743476956912926, 0.1036623340096832, 0.06441389612224711, 0.04273746982819821], 'brevity_penalty': 0.529231618248147, 'length_ratio': 0.6111240087486712, 'translation_length': 100030, 'reference_length': 163682}
rouge_score:  {'rouge1': 0.24450318556912967, 'rouge2': 0.03132887320581654, 'rougeL': 0.11269327554578429, 'rougeLsum': 0.1592316770773422}
bert_score:  0.7734920636695974


In [18]:
res

{'predict': [{'generated_text': [{'role': 'user',
     'content': 'You are a powerful assistant in writing the related works section of a computer science -- AI paper. You have been given a specific topic as title, which covers the high-level concept of a section of related works. You are also given a list of papers that you need to include as references and citations in the related works section. Your task is to write a paragraph that introduces the topic with correct citations. You may comprehend the papers to help you write the paragraph.\nFor each paper, a heterogeneous knowledge graph about the paper is constructed. The graph, provided in text, conveys four important aspects of the paper, namely "research_background", "research_question", "methodology" and "key_results". For each aspect, several keywords are given to provide high-level information. You can use the provided graphs to help you understand the papers and write the paragraph. However, you can also choose to ignore the 

In [9]:
import numpy as np
# calculate the BLEU score with HUGGINGFACE datasets
import evaluate

bleu_score_metric = evaluate.load("bleu")
for id, res in res_dict.items():
    bleu_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bleu_score = bleu_score_metric.compute()
print("bleu_score: ", bleu_score)

rouge_score_metric = evaluate.load("rouge")
for id, res in res_dict.items():
    rouge_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
rouge_score = rouge_score_metric.compute()
print("rouge_score: ", rouge_score)

bert_score_metric = evaluate.load("bertscore")
for id, res in res_dict.items():
    bert_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bert_score = bert_score_metric.compute(model_type="distilbert-base-uncased")
print("bert_score: ", np.mean(bert_score['precision']))

bleu_score:  {'bleu': 0.08927876998456319, 'precisions': [0.1245171334234342, 0.08852481468166333, 0.0789748685179792, 0.07298098846279122], 'brevity_penalty': 1.0, 'length_ratio': 4.362359508634715, 'translation_length': 292265, 'reference_length': 66997}
rouge_score:  {'rouge1': 0.35857489742799686, 'rouge2': 0.23754719828266235, 'rougeL': 0.27778690266208955, 'rougeLsum': 0.2972290725568194}


/hpc2hdd/home/xzou428/miniconda3/envs/yuh/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


bert_score:  0.8776230165362358
